<a href="https://colab.research.google.com/github/brentwilder/alpine3d-modeling/blob/main/get_prism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download PRISM data from GEE



In [ ]:
# Run all of the things needed to get GEE going in colab
!pip install -q earthengine-api
!earthengine authenticate
!pip install -q geemap
from google.colab import auth
auth.authenticate_user()
import ee
ee.Initialize()
import geemap 

In [ ]:
# Define study domain
ulx = -117
uly = 45
lrx = -114
lry = 43
area = ee.Feature(ee.Geometry.Rectangle(ulx,lry,lrx,uly))

In [ ]:
# Call data from earth engine
def clip(image):
  return image.clip(area)

collection = ee.ImageCollection('OREGONSTATE/PRISM/AN81m').select('tmean').filterDate('1981-10-01', '2021-10-01').map(clip)

In [ ]:
# Download the data
collection.aggregate_array('system:index').getInfo()
geemap.ee_export_image_collection_to_drive(collection, folder='get_prism',crs='EPSG:4326',scale=4000)

In [ ]:
# in case I need to cancel jobs
!earthengine task cancel all

In [ ]:
# Convert all the GEE tif outputs to nc
import os
import gdal

inputfolder = './prism_tif'
outputfolder = './prism_nc'

for file in os.listdir('./prism_tif'):
     filename = os.fsdecode(file)
     if filename.endswith('.tif'):
         inputfile = inputfolder+'/'+file
         datestring = file[:-3]
         nctype = 'nc'
         outputfile = outputfolder+'/'+datestring+nctype
         ds = gdal.Translate(outputfile, inputfile, format='NetCDF')